In [2]:
# %pip install pandas pyyaml matplotlib


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 10.2 MB 6.4 MB/s            
     |████████████████████████████████| 189 kB 4.1 MB/s            
     |████████████████████████████████| 8.5 MB 4.5 MB/s            
     |████████████████████████████████| 15.6 MB 4.7 MB/s            
     |████████████████████████████████| 500 kB 4.6 MB/s            
     |████████████████████████████████| 61 kB 580 kB/s            
     |████████████████████████████████| 3.0 MB 123 kB/s            
     |████████████████████████████████| 98 kB 2.7 MB/s            
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see dis

In [1]:

import datetime as dt

import os
import pprint
import pathlib
from pathlib import Path

# Third-party packages
from cycler import cycler
import matplotlib.pyplot as plt
import numpy as np
import psycopg2 as pg
import pandas as pd
import yaml
import psycopg2.extras as extras


#%load_ext lab_black

In [3]:
def load_config():
    """Read the config.yaml configuration file"""
    if not os.path.isfile("config.yaml"):
        raise Exception(
            "config.yaml does not exist. Try running 'switch new scenario' to auto-create it."
        )
    with open("config.yaml") as f:
        return yaml.load(f, Loader=yaml.FullLoader)


def load_dotenv():
    try:
        # Try to load environment variables from .env file using dotenv package.
        # If package is not installed, nothing happens.
        from dotenv import load_dotenv

        load_dotenv()
    except ModuleNotFoundError:
        pass


def connect(schema="switch", connection_env_var="DB_URL"):
    """Connects to the Postgres DB

    This function uses the environment variables to get the URL to connect to the DB. Both
    password and user should be passed directly on the URL for safety purposes.

    Parameters
    ----------
    schema: str Schema of the DB to look for tables. Default is switch
    connection_env_var: The environment variable to use as the connection string

    Returns
    -------
    conn: Database connection object from psycopg2
    """
    # load_dotenv()
    # db_url = os.getenv(connection_env_var)
    db_url = ""
    if db_url is None:
        raise Exception(
            f"Please set the environment variable '{connection_env_var}' to the database URL."
            "The format is normally: postgresql://<user>:<password>@<host>:5432/<database>"
        )

    conn = pg.connect(
        db_url,
        options=f"-c search_path={schema}",
    )

    if conn is None:
        raise SystemExit(
            "Failed to connect to PostgreSQL database."
            "Ensure that the database url is correct, format should normally be:"
            "postgresql://<user>:<password>@<host>:5432/<database>"
        )

    # TODO: Send this to the logger
    print("Connection established to PostgreSQL database.")
    return conn


def get_load_data(
    demand_scenario_id: int,
    force_download=False,
    **kwargs,
):
    """Query the load data from the database"""
    fname = f"load_data-{demand_scenario_id}.csv"

    if not os.path.exists(fname) or force_download:
        df = read_from_db(
            table_name="demand_timeseries",
            where_clause=f"demand_scenario_id = '{demand_scenario_id}'",
            **kwargs,
        )
        df = df.sort_values(["load_zone_id", "raw_timepoint_id"])
        df["date"] = df["timestamp_utc"].dt.strftime("%Y-%m-%d").values
        df.to_csv(fname, index=False)
    else:
        df = pd.read_csv(fname, parse_dates=["timestamp_utc"])
    return df


def insert_to_db(
    table_name: str,
    columns: list,
    values,
    db_conn,
    schema,
    id_column=None,
    id_var=None,
    overwrite=False,
    verbose=None,
    **kwargs,
):
    # Safety check if no DB connection is passed
    if not db_conn:
        raise SystemExit(
            "No connection to DB provided. Check if you passed it correctly"
        )
    # Convert columns to a single string to pass it into the query
    columns = ",".join(columns)

    # Default queries.
    # NOTE: We can add new queries on this section
    search_query = f"""
        select {id_column} from {schema}.{table_name} where {id_column} = {id_var};
    """
    default_query = f"""
        insert into {schema}.{table_name}({columns}) values %s;
    """
    clear_query = f"""
        delete from {schema}.{table_name} where {id_column} = {id_var};
    """

    print(f"+ {table_name}: ")

    # Start transaction with DB
    with db_conn as conn:
        with conn.cursor() as curs:

            # Check if ID is in database
            curs.execute(search_query)
            data = curs.fetchall()

            if data and overwrite:
                if verbose:
                    print(data)
                    print(values)
                print("|  Data exists. Overwritting data")
                curs.execute(clear_query)
                extras.execute_values(curs, default_query, values)
            elif not data:
                print("|  Inserting new data to DB.")
                if verbose:
                    print(values)
                extras.execute_values(curs, default_query, values)
            else:
                raise SystemExit(
                    f"\nValue {id_var} for {id_column} already exists on table {table_name}. Use another one."
                )
    ...


def read_from_db(
    table_name: str,
    db_conn,
    schema,
    where_clause: str = None,
    columns: list = None,
    verbose=False,
    **kwargs,
):
    if not db_conn:
        raise SystemExit(
            "No connection to DB provided. Check if you passed it correctly"
        )

    print(f" | Reading from {table_name}")

    columns = "*" if columns is None else ",".join(columns)
    query = f"""
        SELECT {columns}
        FROM {schema}.{table_name}
        """
    if where_clause is not None:
        query += f" WHERE {where_clause}"
    query += ";"

    if verbose:
        print(query)

    return pd.read_sql_query(query, db_conn)


def get_peak_days(data, freq: str = "MS", verbose: bool = False):
    df = data.copy()

    # Get timestamp of monthly peak
    df = df.set_index("timestamp_utc")
    peak_idx = df.groupby(pd.Grouper(freq="MS")).idxmax()["demand_mw"]

    # Get date of peak timestamp
    datetime_idx = pd.to_datetime(peak_idx.values).strftime("%Y-%m-%d").values

    # Get all days where monthly peak demand is observed
    # peak_days = df.loc[df.date.isin(datetime_idx)]

    # Return dataframe with peak days with hourly resolution
    return datetime_idx


def read_config(fname: str, verbose=None) -> dict:
    """Read yaml configuration file"""

    fpath = pathlib.Path(fname)

    full_path = project_path / fpath

    # Load yaml file with full loader
    with open(full_path) as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

    if verbose:
        pprint.pprint(config)

    return config


def get_project_root() -> Path:
    return Path(".").parent.parent


project_path = get_project_root()
data_path = project_path / "data/raw"
fig_path = project_path / "figs"

In [4]:
SCHEMA = "switch"
OVERWRITE = True

In [5]:
# Start db
db_conn = connect()

Connection established to PostgreSQL database.


In [6]:
query = f"""SELECT * from switch.raw_timepoint;"""
raw_tps = pd.read_sql_query(query, db_conn).set_index("timestamp_utc")
raw_tps.head()

,raw_timepoint_id,raw_timeseries_id
timestamp_utc,,
2011-01-03 08:00:00,8816,1
2011-01-03 09:00:00,8817,1
2011-01-03 10:00:00,8818,1
2011-01-03 11:00:00,8819,1
2011-01-03 12:00:00,8820,1


In [7]:
config = read_config("sampling.yaml", verbose=True)

study_timeframe_id = config["study_timeframe"].get("id")
study_name = config["study_timeframe"].get("name")
study_description = config["study_timeframe"].get("description")
time_sample_id = config["sampling"].get("id")
agg = config["sampling"].get("agg")
ts_name = config["sampling"].get("name")
method = config["sampling"].get("method")
ts_description = config["sampling"].get("description")
demand_scenario_id = config["demand_scenario"]

{'demand_scenario': 174,
 'periods': {2030: {'end_year': 2032, 'length': 5, 'start_year': 2028},
             2035: {'end_year': 2037, 'length': 5, 'start_year': 2033},
             2040: {'end_year': 2042, 'length': 5, 'start_year': 2038},
             2045: {'end_year': 2047, 'length': 5, 'start_year': 2043},
             2050: {'end_year': 2052, 'length': 5, 'start_year': 2048}},
 'sampling': {'description': 'Peak + median day per month with 24-hrs '
                             'resolution for CESM1-CAM5_rcp85 climate '
                             'scenario.\n',
              'id': 38,
              'method': 'M1',
              'name': 'P+M+24hr+CESM1-CAM5_rcp85',
              'tps_per_day': 24},
 'study_timeframe': {'description': '2030-2050 periods with 5 year length '
                                    'each.\n',
                     'id': 39,
                     'name': 2050}}


In [8]:
period_values = [
    (
        study_timeframe_id,
        period_id + 1,
        period_info["start_year"],
        period,
        period_info["length"],
        period_info["end_year"],
    )
    for period_id, (period, period_info) in enumerate(config["periods"].items())
]
period_values

[(39, 1, 2028, 2030, 5, 2032),
 (39, 2, 2033, 2035, 5, 2037),
 (39, 3, 2038, 2040, 5, 2042),
 (39, 4, 2043, 2045, 5, 2047),
 (39, 5, 2048, 2050, 5, 2052)]

In [9]:
no_timepoints = config["sampling"].get("tps_per_day")
# if no_timepoints != 6:
#     raise NotImplementedError("Not yet implemented")
delta_t = int(24 / no_timepoints)
print(f"Number of timepoints per day: {no_timepoints} with {delta_t} hours duration.")

Number of timepoints per day: 24 with 1 hours duration.


In [10]:
fname = "data/load_ts_174.csv"
df = pd.read_csv(
    fname,
    dtype={
        "load_zone_id": "int",
        "demand_scenario_id": "int",
        "load_zone_name": "str",
    },
    parse_dates=["timestamp_utc"],
    usecols=np.r_[:6],
    index_col="timestamp_utc",
)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15340610 entries, 2016-01-01 08:00:00 to 2051-01-01 08:00:00
Data columns (total 5 columns):
 #   Column              Dtype  
---  ------              -----  
 0   load_zone_id        int64  
 1   demand_scenario_id  int64  
 2   raw_timepoint_id    int64  
 3   load_zone_name      object 
 4   demand_mw           float64
dtypes: float64(1), int64(3), object(1)
memory usage: 702.2+ MB


In [11]:
# df[df.load_zone_name.str.startswith("CA_")]

In [12]:
# Aggregate load by zone per timestamp
#data_ca = df[df.load_zone_name.str.startswith("CA_")].copy()
# Aggregate load by zone per timestamp
load_total = (
    df.groupby(["timestamp_utc", "raw_timepoint_id"])[["demand_mw"]]
    .sum()
    .reset_index(level=1)
)
load_total

,raw_timepoint_id,demand_mw
timestamp_utc,,
2016-01-01 08:00:00,52592,-59.279918
2016-01-01 09:00:00,52593,84649.198473
2016-01-01 10:00:00,52594,81357.305873
2016-01-01 11:00:00,52595,79202.762673
2016-01-01 12:00:00,52596,78409.977273
...,...,...
2051-01-01 04:00:00,359404,180179.990807
2051-01-01 05:00:00,359405,192725.399907
2051-01-01 06:00:00,359406,205990.926807


In [13]:
#convert to pacific time
load_total_pst = load_total.tz_localize('utc').tz_convert('US/pacific')

In [14]:
def get_peak_days(data, freq: str = "MS", verbose: bool = False):
    df = data.copy()

    # Get timestamp of monthly peak
    peak_idx = df.groupby(pd.Grouper(freq="MS")).idxmax()["demand_mw"]

    # Get date of peak timestamp
    datetime_idx = pd.to_datetime(peak_idx.values).strftime("%Y-%m-%d").values

    df["date"] = df.index.strftime("%Y-%m-%d")

    # Get all days where monthly peak demand is observed
    peak_days = df.loc[df.date.isin(datetime_idx)]

    # Return dataframe with peak days with hourly resolution
    return datetime_idx, peak_days


def get_next_prv_day(date):
    if not isinstance(date, pd.Timestamp):
        date = pd.to_datetime(date)
    prev_day = date + pd.Timedelta(value=-24, unit="hours")
    next_day = date + pd.Timedelta(value=24, unit="hours")
    return prev_day, next_day


def get_median_days(data, freq: str = "MS", verbose: bool = False):
    median_idx = []
    df = data.copy()

    for _, month_data in df.groupby([pd.Grouper(freq="AS"), pd.Grouper(freq="MS")]):
        daily_mean = month_data.groupby(pd.Grouper(freq="D"))["demand_mw"].mean()

        if len(daily_mean) & 1:
            # If 31 days grab median location
            index_median = daily_mean.loc[daily_mean == daily_mean.median()].index[0]
        else:
            # If 30 or 28 days calculate day closes to the median
            # NOTE: It could be the case that there are two days with the same value.
            # In that case... well...run.
            index_median = (np.abs(daily_mean - daily_mean.median())).idxmin()
        median_idx.append(index_median)

    df["date"] = df.index.strftime("%Y-%m-%d")

    # Get date of median timestamp
    median_idx = pd.to_datetime(median_idx).strftime("%Y-%m-%d")

    # Get all days where monthly peak demand is observed
    median_days = df.loc[df.date.isin(median_idx)]
    return median_idx, median_days

In [15]:
sampled_tps = []
for row in period_values:
    (study_id, period_id, start_year, label, scale_to_period, _end_year) = row
    year_demand = load_total.loc[str(label)].copy()
    peak_idx, peak_days = get_peak_days(year_demand)
    median_idx, median_days = get_median_days(year_demand)
    for peak_day in peak_idx:
        prev_day, next_day = get_next_prv_day(peak_day)
        subset = year_demand.loc[peak_day].copy()
        # subset = year_demand.loc[prev_day:next_day].copy()
        month = subset.index.month.unique()[0]
        day = subset.index.day.unique()[0]
        # subset_peak = subset["demand_mw"].idxmax()
        # lw_interval = subset_peak - pd.Timedelta(value=delta_t * 2, unit="hours")
        # up_interval = subset_peak + pd.Timedelta(
        #     value=delta_t * 2 + delta_t, unit="hours"
        # )
        # tps = year_demand[
        #     subset_peak
        #     - pd.Timedelta(value=delta_t * 2, unit="hours") : subset_peak
        #     + pd.Timedelta(value=(delta_t * 2 + delta_t), unit="hours") : delta_t
        # ].copy()  # raise KeyError("Odd number of timepoints")
        tps = year_demand[
            peak_day
        ].copy()  # raise KeyError("Odd number of timepoints")
        tps.loc[:, "study_timeframe_id"] = study_timeframe_id
        tps.loc[:, "time_sample_id"] = time_sample_id
        tps.loc[:, "id_column"] = f"{label}-{month:>02}-{day:>02}_P"
        tps.loc[:, "period_id"] = period_id
        sampled_tps.append(tps)
    for median_day in median_idx:
        prev_day, next_day = get_next_prv_day(median_day)
        subset = year_demand.loc[median_day].copy()
        month = subset.index.month.unique()[0]
        day = subset.index.day.unique()[0]
        tps = subset[::delta_t].copy()
        tps.loc[:, "study_timeframe_id"] = study_timeframe_id
        tps.loc[:, "time_sample_id"] = time_sample_id
        tps.loc[:, "id_column"] = f"{label}-{month:>02}-{day:>02}_M"
        tps.loc[
            :, "period_id"
        ] = period_id  # raise KeyError("Odd number of timepoints")
        sampled_tps.append(tps)
sampled_tps = pd.concat(sampled_tps).sort_index().reset_index()

In [16]:
subset.index.month.unique()[0]


12

In [17]:
# TODO: Add a new argument to identify groups of days.
sampled_tps.loc[:, "date"] = sampled_tps.timestamp_utc.dt.strftime("%Y-%m-%d").values
sampled_tps.loc[:, "days_in_month"] = sampled_tps.timestamp_utc.dt.days_in_month
sampled_tps["year"] = sampled_tps.timestamp_utc.dt.year
sampled_tps["leap_year"] = sampled_tps["year"] % 4
sampled_tps["days_in_year"] = np.where(sampled_tps["leap_year"] == 0, 366, 365)
# Get first timepoint for each date
sampled_tps.loc[:, "first_timepoint_utc"] = sampled_tps.groupby("id_column")[
    "timestamp_utc"
].transform("first")

# Get last timepoint for each date
# FIXME: This might not work if peak is found in the transition between two dates.
sampled_tps.loc[:, "last_timepoint_utc"] = sampled_tps.groupby("id_column")[
    "timestamp_utc"
].transform("last")

In [18]:
# Scaling for peak day
peak_days = sampled_tps["id_column"].str.endswith("P")
median_days = sampled_tps["id_column"].str.endswith("M")

# data.loc[peak_days, 'ts_scale_to_period'] = data['scale_to_period']

# Duration of days for peak. This is for scaling purposes
peak_duration = 1  # No days
sampled_tps.loc[peak_days, "no_days"] = peak_duration
sampled_tps.loc[median_days, "no_days"] = (
    sampled_tps.loc[median_days, "days_in_month"] - peak_duration
)

sampled_tps.loc[:, "name"] = sampled_tps.timestamp_utc.dt.strftime(f"%Y-%m")
sampled_tps.loc[:, "num_timepoints"] = sampled_tps.groupby("id_column")[
    "raw_timepoint_id"
].transform("count")
sampled_tps.loc[:, "hours_per_tp"] = 1
sampled_tps.loc[:, "no_timeseries"] = sampled_tps.groupby(["period_id"])[
    "name"
].transform("nunique")


# NOTE: If you need to scale the timepoints equally in the period just remove the
# scaling of the number of days in the month.

#scale to period (duration in years of each period) * (24 hours per day * number of days that each time series represents 1 for peak and 27/29/30 for median days) / duration in hours per time point (1) * number of time points per timeseries (24)
sampled_tps.loc[:, "scaling_to_period"] = (
    scale_to_period * (24 * sampled_tps["no_days"])
) / (
    sampled_tps["hours_per_tp"]
    # sampled_tps["no_timeseries"]
    # * sampled_tps["hours_per_tp"]
    * sampled_tps["num_timepoints"]
)

sampled_tps["weight"] = (
    sampled_tps["hours_per_tp"]
    * sampled_tps["num_timepoints"]
    # * sampled_tps["no_timeseries"]
    * sampled_tps["scaling_to_period"]
)
sampled_tps[:12]

,timestamp_utc,raw_timepoint_id,demand_mw,study_timeframe_id,time_sample_id,id_column,period_id,date,days_in_month,year,...,days_in_year,first_timepoint_utc,last_timepoint_utc,no_days,name,num_timepoints,hours_per_tp,no_timeseries,scaling_to_period,weight
0,2030-01-09 00:00:00,175512,111044.495746,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
1,2030-01-09 01:00:00,175513,116211.481894,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
2,2030-01-09 02:00:00,175514,124045.126294,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
3,2030-01-09 03:00:00,175515,126359.020794,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
4,2030-01-09 04:00:00,175516,127127.669994,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
5,2030-01-09 05:00:00,175517,126186.413294,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
6,2030-01-09 06:00:00,175518,124139.709928,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
7,2030-01-09 07:00:00,175519,120053.721328,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
8,2030-01-09 08:00:00,175520,117193.460828,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
9,2030-01-09 09:00:00,175521,116633.712728,39,38,2030-01-09_P,1,2030-01-09,31,2030,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0


In [19]:
output_columns_ts = [
    "sampled_timeseries_id",
    "study_timeframe_id",
    "time_sample_id",
    "period_id",
    "id_column",
    "hours_per_tp",
    "num_timepoints",
    "first_timepoint_utc",
    "last_timepoint_utc",
    "scaling_to_period",
]
sampled_ts = (
    sampled_tps.drop_duplicates(["period_id", "id_column"])
    .reset_index(drop=True)
    .copy()
)
sampled_ts.index = sampled_ts.index.rename("sampled_timeseries_id")
sampled_ts = sampled_ts.reset_index()
sampled_ts

,sampled_timeseries_id,timestamp_utc,raw_timepoint_id,demand_mw,study_timeframe_id,time_sample_id,id_column,period_id,date,days_in_month,...,days_in_year,first_timepoint_utc,last_timepoint_utc,no_days,name,num_timepoints,hours_per_tp,no_timeseries,scaling_to_period,weight
0,0,2030-01-09,175512,111044.495746,39,38,2030-01-09_P,1,2030-01-09,31,...,365,2030-01-09,2030-01-09 23:00:00,1.0,2030-01,24,1,12,5.0,120.0
1,1,2030-01-25,175896,115124.351479,39,38,2030-01-25_M,1,2030-01-25,31,...,365,2030-01-25,2030-01-25 23:00:00,30.0,2030-01,24,1,12,150.0,3600.0
2,2,2030-02-02,176088,117428.608220,39,38,2030-02-02_M,1,2030-02-02,28,...,365,2030-02-02,2030-02-02 23:00:00,27.0,2030-02,24,1,12,135.0,3240.0
3,3,2030-02-17,176448,119263.068202,39,38,2030-02-17_P,1,2030-02-17,28,...,365,2030-02-17,2030-02-17 23:00:00,1.0,2030-02,24,1,12,5.0,120.0
4,4,2030-03-10,176952,116912.080629,39,38,2030-03-10_P,1,2030-03-10,31,...,365,2030-03-10,2030-03-10 23:00:00,1.0,2030-03,24,1,12,5.0,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,115,2050-10-13,357480,149139.650322,39,38,2050-10-13_M,5,2050-10-13,31,...,365,2050-10-13,2050-10-13 23:00:00,30.0,2050-10,24,1,12,150.0,3600.0
116,116,2050-11-09,358128,150572.733850,39,38,2050-11-09_M,5,2050-11-09,30,...,365,2050-11-09,2050-11-09 23:00:00,29.0,2050-11,24,1,12,145.0,3480.0
117,117,2050-11-30,358632,162088.473590,39,38,2050-11-30_P,5,2050-11-30,30,...,365,2050-11-30,2050-11-30 23:00:00,1.0,2050-11,24,1,12,5.0,120.0
118,118,2050-12-14,358968,152694.834257,39,38,2050-12-14_M,5,2050-12-14,31,...,365,2050-12-14,2050-12-14 23:00:00,30.0,2050-12,24,1,12,150.0,3600.0


In [20]:
#verify that the weights and scaling is correct!

#sum of scaling factors * number of time points per time series * duration of time point / duration in years of each period, should add up to 8760 or 8784 for leap years
sampled_ts.groupby("period_id")["scaling_to_period"].sum() * 24 * 1 / 5

period_id
1    8760.0
2    8760.0
3    8784.0
4    8760.0
5    8760.0
Name: scaling_to_period, dtype: float64

In [21]:
output_columns_tps = [
    "raw_timepoint_id",
    "study_timeframe_id",
    "time_sample_id",
    "sampled_timeseries_id",
    "period_id",
    "timestamp_utc",
]

sampled_tps_tms = pd.merge(
    sampled_tps,
    sampled_ts[["sampled_timeseries_id", "period_id", "id_column"]],
    how="right",
    on=["period_id", "id_column"],
)
# sampled_tps = sampled_tps_tms[output_columns_tps]
sampled_tps_tms[output_columns_tps]

,raw_timepoint_id,study_timeframe_id,time_sample_id,sampled_timeseries_id,period_id,timestamp_utc
0,175512,39,38,0,1,2030-01-09 00:00:00
1,175513,39,38,0,1,2030-01-09 01:00:00
2,175514,39,38,0,1,2030-01-09 02:00:00
3,175515,39,38,0,1,2030-01-09 03:00:00
4,175516,39,38,0,1,2030-01-09 04:00:00
...,...,...,...,...,...,...
2875,359131,39,38,119,5,2050-12-20 19:00:00
2876,359132,39,38,119,5,2050-12-20 20:00:00
2877,359133,39,38,119,5,2050-12-20 21:00:00
2878,359134,39,38,119,5,2050-12-20 22:00:00


In [22]:
table_name = "study_timeframe"
columns = ["study_timeframe_id", "name", "description"]
id_column = "study_timeframe_id"

# TODO: Maybe find a better way to do this on a general function for all the tables.
# This works for the moment for each table
values = [(study_id, study_name, study_description)]

# Calling insert function
insert_to_db(
    table_name,
    columns,
    values,
    schema=SCHEMA,
    id_column=id_column,
    id_var=study_id,
    db_conn=db_conn,
    overwrite=OVERWRITE,
)

+ study_timeframe: 
|  Inserting new data to DB.


In [23]:
insert_to_db(
    table_name="period",
    columns=[
        "study_timeframe_id",
        "period_id",
        "start_year",
        "label",
        "length_yrs",
        "end_year",
    ],
    values=period_values,
    schema=SCHEMA,
    id_column="study_timeframe_id",
    id_var=study_id,
    db_conn=db_conn,
    overwrite=OVERWRITE,
)

+ period: 
|  Inserting new data to DB.


In [24]:
values = [(time_sample_id, study_id, ts_name, method, ts_description)]

insert_to_db(
    table_name="time_sample",
    columns=[
        "time_sample_id",
        "study_timeframe_id",
        "name",
        "method",
        "description",
    ],
    schema=SCHEMA,
    values=values,
    id_column="time_sample_id",
    id_var=time_sample_id,
    db_conn=db_conn,
    overwrite=OVERWRITE,
)

+ time_sample: 
|  Inserting new data to DB.


In [25]:
tps_table_name = "sampled_timepoint"
ts_table_name = "sampled_timeseries"

id_column = "time_sample_id"

tps_to_db = sampled_tps_tms[output_columns_tps]
ts_to_db = sampled_ts[output_columns_ts].rename(columns={"id_column": "name"})


insert_to_db(
    ts_table_name,
    ts_to_db.columns,
    [tuple(r) for r in ts_to_db.to_numpy()],
    id_column=id_column,
    schema=SCHEMA,
    id_var=time_sample_id,
    db_conn=db_conn,
    overwrite=OVERWRITE,
)
insert_to_db(
    tps_table_name,
    tps_to_db.columns,
    [tuple(r) for r in tps_to_db.to_numpy()],
    id_column=id_column,
    schema=SCHEMA,
    id_var=time_sample_id,
    db_conn=db_conn,
    overwrite=OVERWRITE,
)

+ sampled_timeseries: 
|  Inserting new data to DB.
+ sampled_timepoint: 
|  Inserting new data to DB.
